In [1]:
import os
os.system('export PYSYN_CDBS=/home/kiran/kurucz_models/grp/hst/cdbs/')
# or
# export PYSYN_CDBS=/media/kiran/New Volume/SEM_6/Internship/kurucz_models/grp/hst/cdbs/
# Run the above line in the command prompt before using this code ... open terminal in folder kurucz_models and type this

import numpy as np
import matplotlib.pyplot as plt
import time
from astropy.io import fits
#import spectres
from scipy.stats import chisquare
from scipy import stats
from astropy import units as u
from astropy import constants as const
import pysynphot as S
import spectres
import csv
import scipy.integrate as integrate

/home/kiran/.local/lib/python3.6/site-packages/pysynphot/locations.py:346: UserWarning: Extinction files not found in /home/kiran/kurucz_models/grp/hst/cdbs/extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))
/home/kiran/.local/lib/python3.6/site-packages/pysynphot/refs.py:118: UserWarning: No graph or component tables found; functionality will be SEVERELY crippled. No files found for /home/kiran/kurucz_models/grp/hst/cdbs/mtab/*_tmg.fits
  'functionality will be SEVERELY crippled. ' + str(e))
/home/kiran/.local/lib/python3.6/site-packages/pysynphot/refs.py:125: UserWarning: No thermal tables found, no thermal calculations can be performed. No files found for /home/kiran/kurucz_models/grp/hst/cdbs/mtab/*_tmt.fits
  'no thermal calculations can be performed. ' + str(e))


In [2]:
def reduced_chi_square(x,y,s,m): # ddof = v
    v = x.size - m
    chi2 = (np.sum((x-y)**2/s**2))/v
    p = 1 - stats.chi2.cdf(chi2, v)
    return chi2,p

def get_photometric_flux(m_wave,interpolated_flux,F):
    for X in list(F.keys()):
        w = []
        Sx = []
        with open("{}_2MASS.txt".format(X),'r') as f:
            for line in f:
                y = [float(i) for i in line.split()]
                w.append(y[0]*(10**(-4))) # for converting A to um
                Sx.append(y[1])
        wave = np.array(list(set().union(w,m_wave)))
        wave = np.sort(wave)
        idx = np.where(np.logical_and(wave>=0.3000, wave<=10.0000))
        wave = wave[idx]
        interpolated_Sx = np.interp(wave, w, Sx)
        interpolated_m_flux = np.interp(wave, m_wave, interpolated_flux)
        F[X] = integrate.simps(interpolated_Sx*interpolated_m_flux, wave)/integrate.simps(interpolated_Sx, wave)

    return F


class LessPrecise(float):
    def __repr__(self):
        return str(self)

In [39]:
J = [1.1,1.4]
H = [1.5,1.8]
Ks = [2.0,2.4]

######### USER INPUTS ########
savefig  = False
filename = 'hd11443'
data_dir = os.getcwd()+"/IRTF_data_fit/"
form = '.txt'
flux_error_data = True
conversion_factor = 10**(-4)
##############################

s_wave = []
s_flux = []
a = np.array([])
token = True
with open(data_dir+filename+'/'+ filename+form,'r') as f:
    for line in f:
        if(len(line.split()) == 3):
            if(a.size == 0):
                try:
                    a = np.array([float(i) for i in line.split()])
                except:
                    wave_unit, flux_unit, _ = line.split()                
            else:
                a = np.row_stack((a,[float(i) for i in line.split()]))
f.close()
m_ref = {"J": 0, "H": 0,"KS": 0}
m_err_ref = {"JERR": 0, "HERR": 0,"KSERR": 0}
ctr = 6
q = 0
with open(data_dir+filename+'/'+ filename+form,'r') as f:
    for line in f:
        if(ctr > 0):
            _b = line.split()
            if(q%2 == 0):
                m_ref[_b[1]] = float(_b[3])
            if(q%2 == 1):
                m_err_ref[_b[1]] = float(_b[3])
            q += 1
            ctr -= 1
f.close()
m_ref["Ks"] = m_ref.pop("KS")
#wvl, flx = pyasl.read1dFitsSpec("_bd+622346_20121027_735_cbuil.fits")
a_uv = np.copy(a)
freq_range = [0.3,10.0]
if(wave_unit == "um"):
    while(a[0][0] <= freq_range[0]):
        a = a[1:]
    while(a.T[0,-1] > freq_range[1]):
        a = a[:-2]
else:
    while(a[0][0] <= freq_range[0]*(10**4)):
        a = a[1:]
    while(a.T[0,-1] > freq_range[1]*(10**4)):
        a = a[:-2]
    a.T[0] *= 10**(-4)
    a_uv.T[0] *= 10**(-4)

del_arr = np.where(a.T[1]<0)
a = np.delete(a,del_arr,0)
del_arr = np.where(a_uv.T[1]<0)
a_uv = np.delete(a_uv,del_arr,0)

#m_flux *= np.amax(s_flux)
s_wave = np.array(a.T[0])
s_flux = np.array(a.T[1])#*10**(-3) # convert ergs/s/cm^2/A -> W/cm^2/um by multiplying by (10**(-7))/(10**(-4))                
s_flux_e = np.array(a.T[2])
s_flux = s_flux*conversion_factor # conversion factor given 
s_flux_e = s_flux_e*conversion_factor # conversion factor given 
a_uv.T[1] = a_uv.T[1]*conversion_factor
a_uv.T[2] = a_uv.T[2]*conversion_factor

best_r = 0
best_log_g = 0
best_teff = 0
best_met = 0
p = 0; # in mas
dp = 0;
with open("parallax_values.txt",'r') as f:
    for line in f:
        _a,b,c = line.split()
        if(_a == filename):
            p = float(b)
            dp = float(c)
#p = 109.54 # mas
#dp = 0.17

D = 1000/p * u.parsec
D = D.to(u.m)
dD = (1000/p**2)*dp * u.parsec
dD = dD.to(u.m)

In [40]:
def get_rchi2(t,g,m,r,s_wave,s_flux,s_flux_e):
    Md = 0
    savefig = False

    chi = np.inf
    p = 0
    try:
        sp = S.Icat('ck04models', t, m, g)
    except:
        return None
    m_wave = sp.wave*(10**(-4)) # converting A -> um
    m_flux = sp.flux*(10**(-3)) # convert ergs/s/cm^2/A -> W/cm^2/um by multiplying by (10**(-7))/(10**(-4))                
    idx = np.where(np.logical_and(m_wave>=0.3, m_wave<=10)) # considering particular frequency range                            
    m_wave = m_wave[idx]
    m_flux = m_flux[idx]

    s_lower_lim = np.argmin(np.abs(s_wave - m_wave[0]))
    s_upper_lim = np.argmin(np.abs(s_wave - m_wave[-1]))


    m_lower_lim = np.argmin(np.abs(m_wave - np.round(s_wave[0])))
    m_upper_lim = np.argmin(np.abs(m_wave - np.round(s_wave[-1])))

    m_wave = m_wave[m_lower_lim:m_upper_lim+1]
    m_flux = m_flux[m_lower_lim:m_upper_lim+1]
    s_wave = s_wave[s_lower_lim:s_upper_lim+1]
    s_flux = s_flux[s_lower_lim:s_upper_lim+1]

    if(flux_error_data):
        if(m_wave.size > s_wave.size):
            s_flux_e = s_flux_e[s_lower_lim:s_upper_lim+1]
            interpolated_flux, interpolated_flux_e = spectres.spectres(m_wave, s_wave, s_flux, s_flux_e)
            Md = ((r*const.R_sun)/D)**2
            interpolated_flux = interpolated_flux*Md
            redchi2_arr.append(reduced_chi_square(s_flux,interpolated_flux,interpolated_flux_e,4)[0])
            if(reduced_chi_square(s_flux,interpolated_flux,interpolated_flux_e,4)[0] < chi):
                chi,p = reduced_chi_square(s_flux,interpolated_flux, interpolated_flux_e,4)
                best_r = r
                best_log_g = g
                best_teff = t
                best_met = m

            s_wave = s_wave[:m_wave.size]
            s_flux = s_flux[:m_wave.size]
        else:
            s_flux_e = s_flux_e[s_lower_lim:s_upper_lim+1]
            interpolated_flux = np.interp(s_wave, m_wave, m_flux)
            Md = ((r*const.R_sun)/D)**2
            interpolated_flux = (interpolated_flux*Md).value
            redchi2_arr.append(reduced_chi_square(s_flux,interpolated_flux,s_flux_e,4)[0])
            if(reduced_chi_square(s_flux,interpolated_flux,s_flux_e,4)[0] < chi):
                chi,p = reduced_chi_square(s_flux,interpolated_flux, s_flux_e,4)
                best_r = r
                best_log_g = g
                best_teff = t
                best_met = m
            m_wave = s_wave
            m_flux = interpolated_flux/Md
    else:
        if(s_wave.size > m_wave.size):
            interpolated_flux = np.interp(s_wave, m_wave, m_flux)
            Md = ((r*const.R_sun)/D)**2
            interpolated_flux = interpolated_flux*Md

            if(chisquare(s_flux,interpolated_flux)[0] < chi):
                chi,p = chisquare(s_flux,interpolated_flux)
                best_r = r
                best_log_g = g
                best_teff = t
                best_met = m
            m_wave = s_wave
            m_flux = interpolated_flux/Md
        else:
            interpolated_flux = np.interp(m_wave, s_wave, s_flux)
            Md = ((r*const.R_sun)/D)**2
            s_flux = np.copy(interpolated_flux)

            if(chisquare(s_flux,interpolated_flux)[0] < chi):
                chi,p = chisquare(s_flux,interpolated_flux)
                best_r = r
                best_log_g = g
                best_teff = t
                best_met = m
            interpolated_flux = m_flux*Md
            s_wave = m_wave
            
    return chi,p

In [41]:
redchi2 = {}
t0,g0,m0,r0 = 6100,3.9,-0.1,3.2#5200,4.5,0.0,0.8
fit_arr = {}
teff = [t0-100,t0,t0+100]
logg = [g0-0.1,g0,g0+0.1]
met = [m0-0.1,m0,m0+0.1]
rad = [r0-0.1,r0,r0+0.1]
redchi2["teff"] = [get_rchi2(t,g0,m0,r0,s_wave,s_flux,s_flux_e)[0] for t in teff]
redchi2["logg"] = [get_rchi2(t0,g,m0,r0,s_wave,s_flux,s_flux_e)[0] for g in logg]
redchi2["met"] = [get_rchi2(t0,g0,m,r0,s_wave,s_flux,s_flux_e)[0] for m in met]
redchi2["rad"] = [get_rchi2(t0,g0,m0,r,s_wave,s_flux,s_flux_e)[0] for r in rad]

def get_optimals(teff,redchi2_arr):
    tmp = teff[1] - (teff[1]-teff[0])*(((redchi2_arr[2]-redchi2_arr[1])/(redchi2_arr[0]-2*redchi2_arr[1]+redchi2_arr[2]))+0.5)
    tmp_err = (teff[1]-teff[0])*np.sqrt(2.0/(redchi2_arr[0]-2*redchi2_arr[1]+redchi2_arr[2]))
    return tmp, tmp_err

fit_arr["teff"] = get_optimals(teff,redchi2["teff"])
fit_arr["logg"] = get_optimals(logg,redchi2["logg"])
fit_arr["met"] = get_optimals(met,redchi2["met"])
fit_arr["rad"] = get_optimals(rad,redchi2["rad"])

# HD11443

In [42]:
redchi2

{'teff': [12591.418395375396, 9678.040117380579, 8653.43534694444],
 'logg': [9625.936904742073, 9678.040117380579, 9730.972194212502],
 'met': [10008.549552747674, 9678.040117380579, 9387.069152832137],
 'rad': [16219.43036033776, 9678.040117380579, 11479.17708411826]}

In [45]:
fit_arr

{'teff': (6104.247095606528, 3.2540562545638814),
 'logg': (-2.5360976565608113, 0.15533642594587935),
 'met': (0.5859186092021658, 0.0224908082813641),
 'rad': (3.1284101757001817, 0.0015483394594384512)}

In [47]:
print(get_rchi2(fit_arr["teff"][0],fit_arr["logg"][0],fit_arr["met"][0],fit_arr["rad"][0],s_wave,s_flux,s_flux_e))

None


# HD10476

In [38]:
redchi2

{'teff': [5834.3754701178295, 3555.104530669988, 3727.683118526266],
 'logg': [3553.5720184654388, 3555.104530669988, 3546.7488402537906],
 'met': [3626.9863473610535, 3555.104530669988, 3447.89508405743],
 'rad': [43601.291359870134, 3555.104530669988, 47159.70336571278]}

In [37]:
fit_arr

{'teff': (5142.961289592431, 2.8560650257762688),
 'logg': (4.365498389983776, nan),
 'met': (-0.35347194773837354, nan),
 'rad': (0.6978730552393582, 0.0004889674906246204)}